Importing Datasets

In [ ]:
import json
import shutil
from pathlib import Path


8K

In [ ]:
#Downloading datasets from kaggle using Tomer's Api from kaggle.
from google.colab import files
files.upload()
!mkdir -p ~/.kaggle
!mv kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json
!kaggle datasets download -d adityajn105/flickr8k
!unzip flickr8k.zip -d flickr8k



Streaming output truncated to the last 5000 lines.
  inflating: flickr8k/Images/2844846111_8c1cbfc75d.jpg  
  inflating: flickr8k/Images/2844963839_ff09cdb81f.jpg  
  inflating: flickr8k/Images/2845246160_d0d1bbd6f0.jpg  
  inflating: flickr8k/Images/2845691057_d4ab89d889.jpg  
  inflating: flickr8k/Images/2845845721_d0bc113ff7.jpg  
  inflating: flickr8k/Images/2846037553_1a1de50709.jpg  
  inflating: flickr8k/Images/2846785268_904c5fcf9f.jpg  
  inflating: flickr8k/Images/2846843520_b0e6211478.jpg  
  inflating: flickr8k/Images/2847514745_9a35493023.jpg  
  inflating: flickr8k/Images/2847615962_c330bded6e.jpg  
  inflating: flickr8k/Images/2847859796_4d9cb0d31f.jpg  
  inflating: flickr8k/Images/2848266893_9693c66275.jpg  
  inflating: flickr8k/Images/2848571082_26454cb981.jpg  
  inflating: flickr8k/Images/2848895544_6d06210e9d.jpg  
  inflating: flickr8k/Images/2848977044_446a31d86e.jpg  
  inflating: flickr8k/Images/2849194983_2968c72832.jpg  
  inflating: flickr8k/Images/28507194

30k

In [ ]:
!kaggle datasets download -d hsankesara/flickr-image-dataset
!unzip flickr-image-dataset.zip -d flickr30k

Streaming output truncated to the last 5000 lines.
  inflating: flickr30k/flickr30k_images/flickr30k_images/flickr30k_images/535529555.jpg  
  inflating: flickr30k/flickr30k_images/flickr30k_images/flickr30k_images/5356236664.jpg  
  inflating: flickr30k/flickr30k_images/flickr30k_images/flickr30k_images/5356318668.jpg  
  inflating: flickr30k/flickr30k_images/flickr30k_images/flickr30k_images/535830521.jpg  
  inflating: flickr30k/flickr30k_images/flickr30k_images/flickr30k_images/5359437945.jpg  
  inflating: flickr30k/flickr30k_images/flickr30k_images/flickr30k_images/5361355399.jpg  
  inflating: flickr30k/flickr30k_images/flickr30k_images/flickr30k_images/53614287.jpg  
  inflating: flickr30k/flickr30k_images/flickr30k_images/flickr30k_images/536147209.jpg  
  inflating: flickr30k/flickr30k_images/flickr30k_images/flickr30k_images/5361551308.jpg  
  inflating: flickr30k/flickr30k_images/flickr30k_images/flickr30k_images/536156200.jpg  
  inflating: flickr30k/flickr30k_images/flick

MSCOCO 2017

In [ ]:

!wget http://images.cocodataset.org/zips/train2017.zip
!unzip train2017.zip -d train2017_images


Streaming output truncated to the last 5000 lines.
 extracting: train2017_images/train2017/000000259556.jpg  
 extracting: train2017_images/train2017/000000381931.jpg  
 extracting: train2017_images/train2017/000000569592.jpg  
 extracting: train2017_images/train2017/000000229396.jpg  
 extracting: train2017_images/train2017/000000488990.jpg  
 extracting: train2017_images/train2017/000000348684.jpg  
 extracting: train2017_images/train2017/000000234031.jpg  
 extracting: train2017_images/train2017/000000563584.jpg  
 extracting: train2017_images/train2017/000000276069.jpg  
 extracting: train2017_images/train2017/000000401194.jpg  
 extracting: train2017_images/train2017/000000502089.jpg  
 extracting: train2017_images/train2017/000000192183.jpg  
 extracting: train2017_images/train2017/000000425127.jpg  
 extracting: train2017_images/train2017/000000126766.jpg  
 extracting: train2017_images/train2017/000000324161.jpg  
 extracting: train2017_images/train2017/000000452746.jpg  
 extr

In [ ]:

!wget http://images.cocodataset.org/annotations/annotations_trainval2017.zip
!unzip annotations_trainval2017.zip -d annotations


--2025-01-07 10:09:15--  http://images.cocodataset.org/annotations/annotations_trainval2017.zip
Resolving images.cocodataset.org (images.cocodataset.org)... 54.231.162.65, 52.217.90.132, 3.5.27.165, ...
Connecting to images.cocodataset.org (images.cocodataset.org)|54.231.162.65|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 252907541 (241M) [application/zip]
Saving to: ‘annotations_trainval2017.zip’

annotations_trainva 100%[===================>] 241.19M  59.3MB/s    in 4.5s    

2025-01-07 10:09:20 (53.4 MB/s) - ‘annotations_trainval2017.zip’ saved [252907541/252907541]

Archive:  annotations_trainval2017.zip
  inflating: annotations/annotations/instances_train2017.json  
  inflating: annotations/annotations/instances_val2017.json  
  inflating: annotations/annotations/captions_train2017.json  
  inflating: annotations/annotations/captions_val2017.json  
  inflating: annotations/annotations/person_keypoints_train2017.json  
  inflating: annotations/annotation

Creating Dataloader

In [ ]:
import os
import json
from PIL import Image
import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from nltk.tokenize import word_tokenize
from collections import Counter
import nltk
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from tqdm import tqdm
import json
import torch.nn as nn
from torchvision import models
import ast


In [ ]:
nltk.download('punkt_tab')
nltk.download('punkt')

class ImageCaptionDataset(Dataset):
    def __init__(self, image_dir, caption_file, transform=None, vocab=None):
        """
        Args:
            image_dir (str): Directory with images.
            caption_file (str): File with captions
            transform (callable): Transform for images.
            vocab (Vocabulary): Vocabulary for tokenizing captions.
        """
        self.image_dir = image_dir
        self.transform = transform
        self.vocab = vocab
        self.feature_cache = {}



        # Load captions
        if caption_file.endswith(".json"):
            with open(caption_file, 'r') as f:
                captions_data = json.load(f)
            self.image_captions = [
                (f"{str(item['image_id']).zfill(12)}.jpg", item['caption'])
                for item in captions_data['annotations']
            ]
        elif caption_file.endswith(".csv"):
            captions_df = pd.read_csv(caption_file, delimiter='\t')
            image_column = captions_df.columns[0]

            caption_column = 'caption'
            self.image_captions = list(zip(captions_df[image_column], captions_df[caption_column]))

        elif caption_file.endswith(".xlsx"):
            captions_df = pd.read_excel(caption_file)
            image_column = captions_df.columns[0]

            caption_column = 'caption'
            self.image_captions = list(zip(captions_df[image_column], captions_df[caption_column]))
        elif caption_file.endswith(".txt"):
            self.image_captions = []
            with open(caption_file, 'r') as f:
                lines = f.readlines()
                for line in lines:
                    image_name, caption = line.strip().split(',',1)
                    image_name = image_name.split('#')[0]  # Handle 'image#n' format
                    self.image_captions.append((image_name, caption))
        else:
            raise ValueError("Unsupported caption file format")


    def __len__(self):
        return len(self.image_captions)

    def __getitem__(self, idx):
        # Get image name and caption
        image_name, caption = self.image_captions[idx]
        image_path = os.path.join(self.image_dir, image_name)
        if image_name.startswith('/content/'):
            image_name = image_name[len('/content/'):]
        # Return image path and the original caption string
        return image_path, caption


[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [ ]:
# Instantiate datasets for Flickr8k, Flickr30k, and MS COCO
flickr8k_dataset = ImageCaptionDataset(image_dir="flickr8k/Images",caption_file="flickr8k/captions.txt")

#coco_dataset = ImageCaptionDataset(image_dir="/content/train2017_images/train2017",caption_file="annotations/annotations/captions_train2017.json")

#flickr30k_dataset = ImageCaptionDataset(image_dir="/content/flickr30k/flickr30k_images/flickr30k_images",caption_file="/content/flickr30k/flickr30k_images/cleaned_flickr30k.xlsx")



In [ ]:
im_dic={}
for i in range(len(flickr8k_dataset)):
    if flickr8k_dataset[i][0] in im_dic:
        im_dic[flickr8k_dataset[i][0]].append(flickr8k_dataset[i][1])
    else:
        im_dic[flickr8k_dataset[i][0]]=[flickr8k_dataset[i][1]]


In [ ]:
im_dic['flickr8k/Images/1580172290_e19067e0dd.jpg']

['A baby and a young boy playing .',
 'A young boy crawls with a toddler .',
 'Two children on hands and knees on a wood floor .',
 'Two kids play on the floor .',
 'two young babies crawling on a wood floor']

In [ ]:
from math import ceil


num_subsets = 10

# Convert the dictionary to a list of items
im_dic_items = list(im_dic.items())

# Calculate the size of each subset
subset_size = ceil(len(im_dic_items) / num_subsets)

# Create the subsets
subsets = [
    dict(im_dic_items[i:i + subset_size])
    for i in range(0, len(im_dic_items), subset_size)
]

# Example: Print subset sizes
for i, subset in enumerate(subsets):
    print(f"Subset {i + 1} size: {len(subset)}")


Subset 1 size: 11829
Subset 2 size: 11829
Subset 3 size: 11829
Subset 4 size: 11829
Subset 5 size: 11829
Subset 6 size: 11829
Subset 7 size: 11829
Subset 8 size: 11829
Subset 9 size: 11829
Subset 10 size: 11826


In [ ]:
dic_use=subsets[9]

In [ ]:
class Image_Dataset(Dataset):
    def __init__(self, im_dic, transform=None):
        self.image_paths = list(im_dic.keys())
        self.captions = list(im_dic.values())
        self.transform = transform

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        image_path = self.image_paths[idx]
        image = Image.open(image_path).convert('RGB')
        if self.transform:
            image = self.transform(image)
        return image, image_path


In [ ]:
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

In [ ]:
dataset = Image_Dataset(dic_use, transform=transform)
data_loader = DataLoader(dataset, batch_size=32, shuffle=False, num_workers=4)
features_dic = {}



/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:617: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(


Loading Images into Tensors

In [ ]:
class EncoderCNN(nn.Module):
    def __init__(self):
        super(EncoderCNN, self).__init__()
        resnet = models.resnet50(pretrained=True)
        # Remove the classification layer
        modules = list(resnet.children())[:-1]  # Remove the last fully connected layer
        self.resnet = nn.Sequential(*modules)

    def forward(self, images):
        with torch.no_grad():
            features = self.resnet(images)  # Extract features
        features = features.view(features.size(0), -1)  # Flatten the feature maps
        return features
'''
with torch.no_grad(): This context manager ensures that no gradients are computed for the ResNet part. Since we’re using a pretrained ResNet as a feature extractor,
 we typically freeze its weights so they are not updated during training.
 Freezing weights:

Reduces computational cost (no gradient calculations).
Prevents the model from “forgetting” the general features it learned.
'''


'\nwith torch.no_grad(): This context manager ensures that no gradients are computed for the ResNet part. Since we’re using a pretrained ResNet as a feature extractor,\n we typically freeze its weights so they are not updated during training.\n Freezing weights:\n\nReduces computational cost (no gradient calculations).\nPrevents the model from “forgetting” the general features it learned.\n'

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
encoder = EncoderCNN().to(device).half()

/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to /root/.cache/torch/hub/checkpoints/resnet50-0676ba61.pth
100%|██████████| 97.8M/97.8M [00:00<00:00, 133MB/s]


In [ ]:
encoder.eval()  # Set the encoder to evaluation mode
for images, image_paths in tqdm(data_loader, desc="Extracting Features"):
    images = images.to(device).half()  # Move images to the target device
    with torch.no_grad():
        features = encoder(images)  # Extract features
    features = features.cpu().numpy()  # Move features to CPU and convert to numpy

    for image_path, feature in zip(image_paths, features):
        features_dic[image_path] = feature.flatten().tolist()

Extracting Features: 100%|██████████| 370/370 [02:14<00:00,  2.76it/s]


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
rounded_feature_dic = {key: np.round(np.array(value), decimals=6).tolist() for key, value in features_dic.items()}


In [ ]:
captions_dic = {}

for image_path, captions in dic_use.items():
    if image_path in rounded_feature_dic:
        features = tuple(rounded_feature_dic[image_path])  # Convert to a tuple to use as a dictionary key
        captions_dic[features] = captions
    else:
        print(f"Warning: No features found for {image_path}")


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
data = []

for features, captions in captions_dic.items():

      data.append({'Captions': captions, 'Features': list(features)})

# Convert the list of dictionaries to a DataFrame
df = pd.DataFrame(data)


In [ ]:
df.head()

,Captions,Features
0,[A small child playing with a video game contr...,"[1.036133, 1.382812, 1.976562, 0.024887, 0.878..."
1,[A living room area with a bunch of furniture ...,"[0.092834, 0.247559, 0.019257, 0.586914, 1.064..."
2,"[A clock on the top of an old building. , A to...","[0.000728, 0.07135, 0.238892, 0.014778, 0.0393..."
3,[A pair of children play with kites in a park ...,"[0.30127, 0.356934, 0.47168, 0.666992, 0.50683..."
4,[A baseball player is attempting to hit the ba...,"[0.22583, 1.850586, 0.709473, 0.043945, 0.7197..."


In [ ]:
df.to_csv('coco_pat_10.csv', index=False)

Building RNN Model

In [ ]:
coco_1=pd.read_csv('coco_pat_1.csv')
coco_2=pd.read_csv('coco_pat_2.csv')
coco_3=pd.read_csv('coco_pat_3.csv')
coco_4=pd.read_csv('coco_pat_4.csv')
coco_5=pd.read_csv('coco_pat_5.csv')
coco_6=pd.read_csv('coco_pat_6.csv')
coco_7=pd.read_csv('coco_pat_7.csv')
coco_8=pd.read_csv('coco_pat_8.csv')
coco_9=pd.read_csv('coco_pat_9.csv')
coco_10=pd.read_csv('coco_pat_10.csv')
flickr_8k=pd.read_csv('flicker8K_better.csv')
flickr_30k=pd.read_csv('flicker30K_better.csv')
train_data=pd.concat([coco_1,coco_2,coco_3,coco_4,coco_5,coco_6,coco_7,coco_8,coco_9,coco_10,flickr_30k], ignore_index=True)
train_data.head()

FileNotFoundError: [Errno 2] No such file or directory: 'coco_pat_1.csv'

In [ ]:
train_data=pd.read_csv('drive/MyDrive/flicker8K_better.csv')

In [ ]:
failed_rows = []
def safe_literal_eval(val,row_index):
    try:
        return ast.literal_eval(val)
    except (ValueError, SyntaxError):
        failed_rows.append(row_index)
        return [val]  # Return an empty list for malformed data

train_data.iloc[:, 0] = [
    safe_literal_eval(val, idx) for idx, val in enumerate(train_data.iloc[:, 0])
]

In [ ]:
failed_rows

[]

In [ ]:
train_data = train_data.drop(index=failed_rows).reset_index(drop=True)

In [ ]:
captions=train_data['Captions'].tolist()

In [ ]:
captions_lst=[]
for i in captions:
    for j in i:
        captions_lst.append(j)
print(captions_lst[0])

A child in a pink dress is climbing up a set of stairs in an entry way .


In [ ]:
import re
from collections import Counter

class Vocabulary:
    def __init__(self, freq_threshold=1):
        """
        Initializes the Vocabulary object.

        Args:
            freq_threshold (int): Minimum frequency for a word to be included in the vocabulary.
        """
        self.freq_threshold = freq_threshold
        self.word2idx = {}
        self.idx2word = {}
        self.special_tokens = ["<pad>", "<start>", "<end>", "<unk>"]
        self._build_special_tokens()

    def _build_special_tokens(self):
        """
        Adds special tokens to the vocabulary with reserved indices.
        """
        for idx, token in enumerate(self.special_tokens):
            self.word2idx[token] = idx
            self.idx2word[idx] = token

    def build_vocab(self, captions):
        """
        Builds the vocabulary from a list of captions.

        Args:
            captions (list of str): List of captions (each caption is a string).
        """
        token_counter = Counter()

        # Tokenize captions and count frequencies
        for caption in captions:
            tokens = self.tokenize(caption)
            token_counter.update(tokens)

        # Filter out rare words and assign indices
        for word, freq in token_counter.items():
            if freq >= self.freq_threshold and word not in self.word2idx:
                idx = len(self.word2idx)
                self.word2idx[word] = idx
                self.idx2word[idx] = word

    def tokenize(self, text):
        """
        Tokenizes text by splitting on whitespace and removes punctuation.

        Args:
            text (str): Input text to tokenize.

        Returns:
            list of str: List of tokens (words).
        """
        # Remove punctuation using regex
        text = re.sub(r'[^\w\s]', '', text)  # Removes punctuation, keeps words and spaces
        return text.lower().split()  # Tokenizes by whitespace

    def numericalize(self, text):
        """
        Converts text into a sequence of token indices.

        Args:
            text (str): Input text.

        Returns:
            list of int: List of token indices.
        """
        tokens = self.tokenize(text)
        return [self.word2idx.get(token, self.word2idx["<unk>"]) for token in tokens]



# Create a Vocabulary object
vocab = Vocabulary(freq_threshold=1)  # Adjust freq_threshold as needed
vocab.build_vocab(captions_lst)

# Check the generated vocabulary
print("Vocabulary Size:", len(vocab.word2idx))
print("Sample Word to Index Mapping:", {k: vocab.word2idx[k] for k in list(vocab.word2idx)[:10]})




NameError: name 'captions_lst' is not defined

In [ ]:
# Load the vocabulary from the file
with open('vocab.json', 'r') as f:
    loaded_vocab_data = json.load(f)

# Recreate the Vocabulary object
loaded_vocab = Vocabulary(freq_threshold=loaded_vocab_data["freq_threshold"])

# Restore the word2idx and idx2word mappings
loaded_vocab.word2idx = loaded_vocab_data["word2idx"]
loaded_vocab.idx2word = {int(k): v for k, v in loaded_vocab_data["idx2word"].items()}  # Convert keys to int

print("Vocabulary reloaded successfully!")
print("Vocabulary Size:", len(loaded_vocab.word2idx))
print("Sample Word to Index Mapping:", {k: loaded_vocab.word2idx[k] for k in list(loaded_vocab.word2idx)[:10]})


Vocabulary reloaded successfully!
Vocabulary Size: 35474
Sample Word to Index Mapping: {'<pad>': 0, '<start>': 1, '<end>': 2, '<unk>': 3, 'a': 4, 'bicycle': 5, 'replica': 6, 'with': 7, 'clock': 8, 'as': 9}


In [ ]:
# Load the vocabulary from the file
with open('vocab_big.json', 'r') as f:
    loaded_vocab_data = json.load(f)

# Recreate the Vocabulary object
loaded_vocab = Vocabulary(freq_threshold=loaded_vocab_data["freq_threshold"])

# Restore the word2idx and idx2word mappings
loaded_vocab.word2idx = loaded_vocab_data["word2idx"]
loaded_vocab.idx2word = {int(k): v for k, v in loaded_vocab_data["idx2word"].items()}  # Convert keys to int

print("Vocabulary reloaded successfully!")
print("Vocabulary Size:", len(loaded_vocab.word2idx))
print("Sample Word to Index Mapping:", {k: loaded_vocab.word2idx[k] for k in list(loaded_vocab.word2idx)[:10]})


Vocabulary reloaded successfully!
Vocabulary Size: 45090
Sample Word to Index Mapping: {'<pad>': 0, '<start>': 1, '<end>': 2, '<unk>': 3, 'A': 4, 'bicycle': 5, 'replica': 6, 'with': 7, 'a': 8, 'clock': 9}


In [ ]:
class DecoderRNN(nn.Module):
    def __init__(self, feature_dim, hidden_dim, vocab_size, num_layers=2):
        """
        Args:
            feature_dim (int): Dimension of the feature vector from the encoder (e.g., 2048 for ResNet50).
            hidden_dim (int): Dimension of the hidden state in the LSTM.
            vocab_size (int): Size of the output vocabulary.
            num_layers (int): Number of layers in the LSTM.
        """
        super(DecoderRNN, self).__init__()

        # We'll use the feature_dim for the word embedding dimension as well.

        self.embedding = nn.Embedding(vocab_size, feature_dim)

        # LSTM takes in sequences of shape (batch, time_step, input_dim).
        # Our input dimension here will be feature_dim, since we are feeding the
        # image features as the first token and word embeddings for subsequent tokens.


        self.lstm = nn.LSTM(feature_dim, hidden_dim, num_layers=num_layers, batch_first=True)

        # Final fully connected layer maps hidden states to the vocabulary space.
        self.fc = nn.Linear(hidden_dim, vocab_size)

        self.feature_dim = feature_dim

    def forward(self, features, captions):
        """
        Forward pass of the decoder.

        Args:
            features (torch.Tensor): Image features of shape (batch_size, feature_dim).
            captions (torch.Tensor): Tokenized captions of shape (batch_size, caption_length).

        Returns:
            outputs (torch.Tensor): Predicted words for each time step, shape (batch_size, caption_length, vocab_size).
        """
        # Embed the captions (excluding the last word because we predict next word)


        embeddings = self.embedding(captions)  # (batch_size, caption_length, feature_dim)

        # Insert image features at the beginning of the sequence
        # features: (batch_size, feature_dim) -> (batch_size, 1, feature_dim)
        features = features.unsqueeze(1)



        # Concatenate features and embeddings along the time dimension
        inputs = torch.cat((features, embeddings), dim=1)  # (batch_size, 1 + caption_length, feature_dim)



        # Pass through LSTM
        hiddens, _ = self.lstm(inputs)  # (batch_size, 1 + caption_length, hidden_dim)

        # Project the LSTM outputs to the vocabulary space
        outputs = self.fc(hiddens[:,:,:])  # (batch_size, 1 + caption_length, vocab_size)

        return outputs

    def generate_step(self, inputs, states):
       if inputs.dim() == 2:
            inputs = inputs.unsqueeze(1)

        # Pass through LSTM
       hiddens, states = self.lstm(inputs, states)  # hiddens: (batch, 1, hidden_dim)

        # Pass the output through the fully connected layer
       logits = self.fc(hiddens.squeeze(1))  # logits: (batch, vocab_size)

        # Predict the next word
       word_idx = torch.argmax(logits, dim=1).item()  # Get the index of the highest scoring word

        # Prepare the embedding for the next step
       next_input = self.embedding(torch.tensor([word_idx], device=inputs.device))  # (batch=1, feature_dim)

       return {
            "word_idx": word_idx,
            "states": states,
            "next_input": next_input}





In [ ]:
import torch
from torch.utils.data import Dataset, DataLoader
import nltk
import ast
nltk.download('punkt_tab')

class CaptionDataset(Dataset):
    def __init__(self, df, word_to_idx, vocab=None, transform_caption=None):
        """
        Args:
            df (DataFrame): Must have columns ["Features", "Caption"]
            word_to_idx (dict): Maps words to integer indices.
            transform_caption (callable, optional): An optional function to transform captions.
        """
        self.df = df.reset_index(drop=True)
        self.word_to_idx = word_to_idx
        self.idx_to_word = {v: k for k, v in word_to_idx.items()}  # To map indices back to words if needed
        self.transform_caption = transform_caption if transform_caption else self.default_transform
        self.vocab=vocab
    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        features_str = self.df.loc[idx, "Features"]  # This should be a vector (e.g., list or np.array)
        captions = self.df.loc[idx, "Captions"]   # This should be a list of all the lists of the captions

        # Parse string features into a Python list
        features = ast.literal_eval(features_str)
        # Convert features to tensor
        if not torch.is_tensor(features):
            features = torch.tensor(features, dtype=torch.float32)

        caption_tensors = []
        for caption in captions:
            caption_indices = self.transform_caption(caption)
            caption_tensor = torch.tensor(caption_indices, dtype=torch.long)
            caption_tensors.append(caption_tensor)

        return features, caption_tensors

    def default_transform(self, caption):
        # Tokenize caption
        if self.vocab:
            return [self.word_to_idx["<start>"]] + self.vocab.numericalize(caption) + [self.word_to_idx["<end>"]]
        else:
          tokens = nltk.word_tokenize(caption.lower())

        # Map words to indices, use <unk> if not in vocab
        caption_indices = [self.word_to_idx["<start>"]]
        for word in tokens:
            caption_indices.append(self.word_to_idx.get(word, self.word_to_idx["<unk>"]))
        caption_indices.append(self.word_to_idx["<end>"])

        return caption_indices


[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


In [ ]:
import random
def collate_fn(batch):
    """Custom collate function to pad captions."""
    # batch is a list of (features, caption_tensor) tuples
    batch_features = [item[0] for item in batch]
    batch_captions = [item[1] for item in batch]

    # Stack all features
    features = torch.stack(batch_features, dim=0)
    cap_index=random.randint(0,4)
    # Pad captions
    lengths = [len(cap[cap_index]) for cap in batch_captions] # Get the length of the first caption in the list of captions
    max_len = max(lengths)
    padded_captions = torch.ones(len(batch_captions), max_len, dtype=torch.long) * word_to_idx["<pad>"]
    for i, cap_list in enumerate(batch_captions):
        cap = cap_list[cap_index]  # Get a random caption from the list
        end = lengths[i]
        padded_captions[i, :end] = torch.tensor(cap[:end])  # Convert cap[:end] to a tensor


    return features, padded_captions, lengths

In [ ]:
word_to_idx = loaded_vocab.word2idx
dataset_features = CaptionDataset(train_data, vocab=loaded_vocab, word_to_idx=word_to_idx)
train_set, val_set=torch.utils.data.random_split(dataset_features, [int(0.8*len(dataset_features)), len(dataset_features)-int(0.8*len(dataset_features))])
data_loader_features = DataLoader(train_set, batch_size=32, shuffle=True, collate_fn=collate_fn)
val_loader_features = DataLoader(val_set, batch_size=32, shuffle=False, collate_fn=collate_fn)

NameError: name 'train_data' is not defined

In [ ]:
import os
from tqdm import tqdm

# Device configuration
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Model, loss, optimizer, and data loader
decoder = DecoderRNN(feature_dim=2048, hidden_dim=512, vocab_size=len(word_to_idx)).to(device)
criterion = torch.nn.CrossEntropyLoss(ignore_index=word_to_idx["<pad>"])
optimizer = torch.optim.Adam(decoder.parameters(), lr=0.001)
data_loader = data_loader_features
val_loader = val_loader_features
num_epochs = 100

# Path to save checkpoints
checkpoint_dir = "checkpoints"
os.makedirs(checkpoint_dir, exist_ok=True)
checkpoint_path = os.path.join(checkpoint_dir, "decoder_checkpoint.pth")

# Optionally load from a checkpoint
start_epoch = 0
#if os.path.exists(checkpoint_path):
    #checkpoint = torch.load(checkpoint_path)
    #decoder.load_state_dict(checkpoint["model_state"])
    #optimizer.load_state_dict(checkpoint["optimizer_state"])
    #start_epoch = checkpoint["epoch"] + 1
    #print(f"Resumed training from epoch {start_epoch}")

# Training loop
for epoch in range(start_epoch, num_epochs):
    decoder.train()
    total_loss = 0.0

    with tqdm(total=len(data_loader), desc=f"Epoch [{epoch+1}/{num_epochs}]", unit="batch") as pbar:
        for features, captions, lengths in data_loader:
            features = features.to(device)
            captions = captions.to(device)

            # Forward pass
            outputs = decoder(features, captions[:, :-1])
            targets = captions[:, :]

            outputs = outputs[:, :, :]
            loss = criterion(outputs.reshape(-1, outputs.size(-1)), targets.reshape(-1))

            # Backward pass and optimization
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            total_loss += loss.item()
            pbar.set_postfix(loss=loss.item())
            pbar.update(1)

    # Average loss for the epoch
    avg_loss = total_loss / len(data_loader)
    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {avg_loss:.4f}")

# Validation loop
    decoder.eval()
    total_val_loss = 0.0

    with torch.no_grad():  # Disable gradient computation during validation
        for features, captions, lengths in val_loader:
            features = features.to(device)
            captions = captions.to(device)

            # Forward pass
            outputs = decoder(features, captions[:, :-1])
            targets = captions[:, :]

            outputs = outputs[:, :, :]
            loss = criterion(outputs.reshape(-1, outputs.size(-1)), targets.reshape(-1))
            total_val_loss += loss.item()

    # Average validation loss for the epoch
    avg_val_loss = total_val_loss / len(val_loader)
    print(f"Epoch [{epoch+1}/{num_epochs}], Validation Loss: {avg_val_loss:.4f}")

    # Save checkpoint after each epoch
    torch.save({
        "epoch": epoch,
        "model_state": decoder.state_dict(),
        "optimizer_state": optimizer.state_dict(),
        "loss": avg_loss,
    }, checkpoint_path)
    print(f"Checkpoint saved for epoch {epoch+1}")


NameError: name 'word_to_idx' is not defined

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
decoder = DecoderRNN(feature_dim=2048, hidden_dim=512, vocab_size=len(word_to_idx)).to(device)
torch.save(decoder.state_dict(), "decoder_final.pth")

In [ ]:
def generate_caption(decoder, feature_vector, word_to_idx, idx_to_word, max_length=25, verbose=False):
    """
    Generates a caption from a single image feature vector using the trained decoder.

    Args:
        decoder (nn.Module): The trained decoder model.
        feature_vector (list or np.array): Precomputed feature for the image (e.g., 2048-dim).
        word_to_idx (dict): Maps words to integer indices.
        idx_to_word (dict): Maps integer indices back to words.
        max_length (int): Maximum caption length before stopping if <end> not encountered.

    Returns:
        str: The generated caption (excluding <start> and <end> tokens).
    """
    # Convert feature_vector to a tensor on the correct device
    device = next(decoder.parameters()).device
    feature_tensor = torch.tensor(feature_vector, dtype=torch.float32).unsqueeze(0).to(device)

    # Switch model to eval mode (if not already)
    decoder.eval()

    # We'll store generated word indices here
    sampled_indices = []

    # Initialize input to the LSTM.
    # Many "Show & Tell" style decoders require the feature as the first input token.
    # If your decoder is written as decoder(features, captions), you might do a step-by-step generation:
    states = None

    # Expand dims to [batch=1, time=1]
    # We'll feed the feature as the "first token" input
    inputs = feature_tensor  # shape (1, feature_dim)

    # For each word in max_length:
    for _ in range(max_length):
        # Forward pass through the decoder.
        # For an LSTM expecting (features, embeddings)
        outputs = decoder.generate_step(inputs, states)

        word_idx = outputs["word_idx"]     # The ID of the predicted word
        states = outputs["states"]         # Updated LSTM states
        inputs = outputs["next_input"]     # Typically an embedding for the next step

        sampled_indices.append(word_idx)
        if verbose:
            print(f"Step {i}: {idx_to_word.get(word_idx, '<unk>')}")
        if word_idx == word_to_idx["<end>"]:
            break

    # Convert word indices to actual words
    sampled_words = [idx_to_word[idx] for idx in sampled_indices if idx in idx_to_word]

    # Remove <start> and <end> and <pad> if they appear
    caption_tokens = [w for w in sampled_words if w not in ("<start>", "<end>", "<pad>")]
    return " ".join(caption_tokens)

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
decoder = DecoderRNN(feature_dim=2048, hidden_dim=512, vocab_size=len(word_to_idx))
decoder.load_state_dict(torch.load('decoder_final.pth',map_location=torch.device('cpu')))
#decoder.load_state_dict(torch.load('decoder_final_1.pth',map_location=torch.device('cpu')))
decoder.to(device)
decoder.eval()

idx_to_word = {v: k for k, v in word_to_idx.items()}

<ipython-input-9-b39b5bda5b67>:3: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  decoder.load_state_dict(torch.load('decoder_final.pth',map_location=torch.device('cpu')))


In [ ]:
import random
import ast

def test_decoder_on_df(decoder, df, word_to_idx, idx_to_word, samples=5):
    """
    Pick samples random rows from df and generate captions using the decoder.
    Print out the predicted caption and the ground-truth caption for comparison.

    Args:
        decoder (nn.Module): The trained decoder model.
        df (pd.DataFrame): DataFrame with columns ["Features", "Caption"].
        word_to_idx (dict): Maps words to integer indices.
        idx_to_word (dict): Maps integer indices back to words.
        samples (int): How many random examples to test.
    """
    # Switch decoder to eval mode
    decoder.eval()

    # If you want to sample a random subset:
    sampled_df = df.sample(n=samples).reset_index(drop=True)

    for i in range(len(sampled_df)):
        features_str = sampled_df.loc[i, "Features"]
        caption_str  = sampled_df.loc[i, "Captions"]

        # Parse string features into a Python list
        feature_list = ast.literal_eval(features_str)

        # Generate a caption
        predicted_caption = generate_caption(decoder, feature_list, word_to_idx, idx_to_word, max_length=20)

        # Print results
        print(f"Ground Truth: {caption_str}")
        print(f"Predicted   : {predicted_caption}")
        print("-"*50)

In [ ]:
print(len(idx_to_word))

35474


In [ ]:
#flickr_8k=pd.read_csv('flicker8K_better.csv')
flickr_8k=pd.read_csv('flicker8K_better.csv')
test_decoder_on_df(decoder,flickr_8k,word_to_idx,idx_to_word)

Ground Truth: ['A black dog clutches a pink purse in his teeth and runs .', 'A black dog is running on the grass with a pink purse in its mouth .', 'A brown dog retrieves a pink toy .', 'A little black dog is running in the grass with a pink purse in its mouth .', 'The wet dog has retrieved the pink purse with aqua handles .']
Predicted   : a black dog is running through a field
--------------------------------------------------
Ground Truth: ['A baseball player crouching to catch the ball as a runner slides to safety .', 'A boy is sliding towards a base while the baseman is waiting for the ball in a baseball game .', 'Little league slides into plate where rival player crouches in anticipation .', 'Two boys play in a little league game .', 'Two Children playing baseball at the park .']
Predicted   : a baseball player swinging a bat at a ball
--------------------------------------------------
Ground Truth: ['Two guys with one guy holding a homeless sign .', 'Two homeless men are leaning

In [ ]:
test_decoder_on_df(decoder,flickr_8k,word_to_idx,idx_to_word)

Ground Truth: ['A woman biking along a trail surrounded by various plants .', 'A woman rides a bike on a trail through a field .', 'A woman rides a bike over a dirt path through the long grass .', 'A woman riding a bicycle in a field .', 'Woman on bicycle riding down dirt trail .']
Predicted   : a man in a red shirt is standing in front of a large rock wall
--------------------------------------------------
Ground Truth: ['Three girls are walking across a bridge covered with wood and surrounded by trees .', 'Three girls walking along an arched garden pathway .', 'Three girls walk under an arbor on a path surrounded by trees .', 'Three women walking through a tunnel in a park .', 'Three women walk through a plant covered atrium hallway .']
Predicted   : a boy in a red shirt is sitting on a bench outside a building with a man in a
--------------------------------------------------
Ground Truth: ['A couple sits on a floral couch while an older woman in the background smiles .', 'A couple 

In [ ]:
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])


In [ ]:
def Image_to_Caption(image_path,transform=transform,decoder=decoder,word_to_idx=word_to_idx, idx_to_word=idx_to_word):
  image= Image.open(image_path).convert('RGB')
  image=transform(image)
  device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
  encoder = EncoderCNN().to(device)
  encoder.eval()  # Set the encoder to evaluation mode

  image = image.unsqueeze(0).to(device)  # Move images to the target device
  with torch.no_grad():
      features = encoder(image)  # Extract features
  features = features.cpu().numpy()  # Move features to CPU and convert to numpy
  print(generate_caption(decoder,features,word_to_idx,idx_to_word))



In [ ]:
Image_to_Caption('tomer_michal.jpeg')

a man and a woman are sitting on a bus


In [ ]:
decoder.fc

Linear(in_features=512, out_features=35474, bias=True)